In [1]:
import os
from DSSCircuit_Interface import * 
import networkx as nx

In [2]:
FolderName=os.path.dirname(os.path.realpath("__file__"))
d_obj= DSS(r""+ FolderName+ "\Master.dss")  #creating a DSS object instance

In [3]:
d_obj.compile_ckt_dss() #compiling the circuit
d_obj.solve_snapshot_dss(1.0) #solving snapshot power flow

In [4]:
edges_dictlist=[]
G_original=nx.Graph()
for e in d_obj.get_AllPDElements():
    if ((e.split('.')[0]=='Line') or (e.split('.')[0]=='Transformer')): #only if its a line or transformer used in graph(capacitors avoided)
        branch_obj=Branch(d_obj,e) #creating  a branch object instance with full name
        sr_node=branch_obj.bus_fr.split('.')[0] #extracting source bus of branch
        tar_node=branch_obj.bus_to.split('.')[0] #extracting target bus of branch
        name=e.split('.')[1] #getting the name of the element(label or id)
        el_type=e.split('.')[0] #getting the type of the element (line or transformer)
        numphases=branch_obj.nphases #extracting number of phases of branch
        Inorm=branch_obj.Cap #extracting the baseflow current flow in branch
        Imax=branch_obj.MaxCap #extracting the maximum permissible current flow in branch
        Iresid=Imax-Inorm #residual current flow in branch
        if el_type== 'Line':
            R,X=d_obj.get_Line_RX(name)
        else:
            R,X=d_obj.get_Transformer_RX(name)
        edges_dictlist.append({"source":sr_node, "target":tar_node, "label":name, "device":el_type, "resistance":R, "reactance":X, "phases":numphases,"maxcap":Imax,"cap":Inorm,"residue":Iresid})
        G_original.add_edge(sr_node, tar_node, label=name, device=el_type, resistance=R, reactance=X, phases=numphases,maxcap=Imax,cap=Inorm,residue=Iresid)

In [5]:
buses=list(d_obj.get_AllBuses())
loads=d_obj.get_AllLoads()
buses.remove('sourcebus') #sourcebus not in 8500

In [6]:
for b in buses:
    P_sum=0
    Q_sum=0
    for j in range(len(loads)):
        if loads[j]["Bus"]==b:
            P_sum=P_sum+loads[j].get("Pload")
            Q_sum=Q_sum+loads[j].get("Qload")
    G_original.nodes[b]["Device"]="Bus"
    G_original.nodes[b]["ActiveLoad"]=P_sum
    G_original.nodes[b]["ReactiveLoad"]=Q_sum

In [7]:
G_original.edges(data=True)

EdgeDataView([('_hvmv_sub_lsb', 'hvmv_sub_48332', {'label': 'hvmv_sub_connector', 'device': 'Line', 'resistance': 1e-06, 'reactance': 9.999999999999999e-06, 'phases': 3, 'maxcap': 600.0, 'cap': 519.6659824811993, 'residue': 80.33401751880069}), ('_hvmv_sub_lsb', 'regxfmr_hvmv_sub_lsb', {'label': 'feeder_regc', 'device': 'Transformer', 'resistance': 0.0009425454545454546, 'reactance': 0.18850909090909093, 'phases': 1, 'maxcap': 5729.166666666667, 'cap': 502.70382548185904, 'residue': 5226.462841184808}), ('hvmv_sub_48332', 'd5710794-3_int', {'label': 'ln5710794-3', 'device': 'Line', 'resistance': 0.0030582314252429995, 'reactance': 0.0071560770870680015, 'phases': 3, 'maxcap': 600.0, 'cap': 519.6659825076923, 'residue': 80.33401749230768}), ('m1009763', 'l2673322', {'label': 'ln5502549-1', 'device': 'Line', 'resistance': 0.01796107068886, 'reactance': 0.010023346961759999, 'phases': 1, 'maxcap': 600.0, 'cap': 2.1684452866989954, 'residue': 597.831554713301}), ('m1009763', 'l2689691', {'

In [8]:
nx.readwrite.gml.write_gml(G_original,"8500busEx.gml")